# IIoT Data Analytics Notebook (Spark)
Demonstration of [AWS IoT Analytics](https://aws.amazon.com/iot-analytics/) Notebooks, using real-time sensor data.

In [1]:
import boto3
import json
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters

In [2]:
# constants
MY_TIMEZONE='US/Eastern'
DATETIME_FORMAT='%y-%m-%d %H:%M'

In [3]:
def parse(x):
    x = pd.to_datetime(x, infer_datetime_format=True, unit='s', utc=True)
    x = x.tz_convert(MY_TIMEZONE)
    return x

In [4]:
%%time

client = boto3.client('iotanalytics')
dataset = 'iot_analytics_data_set'
data_location = client.get_dataset_content(datasetName=dataset)['entries'][0]['dataURI']
df = pd.read_csv(data_location,
                 header=0,
                 low_memory=False,
                 infer_datetime_format=True,
                 date_parser=parse,
                 index_col=['ts'])

CPU times: user 76.2 ms, sys: 20.9 ms, total: 97.1 ms
Wall time: 340 ms


In [5]:
# clean up DataFrame
df = df.drop(columns='__dt')
df = df.sort_values(by='ts', ascending=True)

In [6]:
# transform metadata to valid json object
df['metadata'].replace('{','{"',regex=True, inplace = True)
df['metadata'].replace('=','":"',regex=True, inplace = True)
df['metadata'].replace(', ','","',regex=True, inplace = True)
df['metadata'].replace(':"{"',':{"',regex=True, inplace = True)
df['metadata'].replace('}",','},',regex=True, inplace = True)
df['metadata'].replace('}','"}',regex=True, inplace = True)

In [10]:
print(df.head(5))

                                            device  humidity   temp  light  \
ts                                                                           
2020-07-11 15:38:57.142367125-04:00  iot-device-02      51.9  71.06  False   
2020-07-11 15:38:57.142367125-04:00  iot-device-02      51.9  71.06  False   
2020-07-11 15:38:57.142367125-04:00  iot-device-02      51.9  71.06  False   
2020-07-11 15:38:57.142367125-04:00  iot-device-02      51.9  71.06  False   
2020-07-11 15:38:57.142367125-04:00  iot-device-02      51.9  71.06  False   

                                     motion       lpg        co     smoke  \
ts                                                                          
2020-07-11 15:38:57.142367125-04:00   False  0.007664  0.004968  0.020449   
2020-07-11 15:38:57.142367125-04:00   False  0.007664  0.004968  0.020449   
2020-07-11 15:38:57.142367125-04:00   False  0.007664  0.004968  0.020449   
2020-07-11 15:38:57.142367125-04:00   False  0.007664  0.004968  0.0

In [11]:
metadata = pd.read_json(df['metadata'][0], orient='records', typ='series')
print(metadata['thingname'])

iot-device-02


In [12]:
# filter time range
#df_filtered = df.loc[df.index >= pd.Timestamp('today', tz=MY_TIMEZONE).floor('D') + pd.Timedelta(-1, unit='D')]
df_filtered = df.loc[df.index >= pd.Timestamp('today', tz=MY_TIMEZONE) + pd.Timedelta(-1, unit='D')] # last 24 hours
#df_filtered = df.tail(10000) # fixed amount of records

In [13]:
# filter temp/humidity outliers (>1% & <99%)
df_filtered = df_filtered.loc[df_filtered['temp'] > df_filtered.groupby('device').temp.transform(lambda x: x.quantile(.01))]
df_filtered = df_filtered.loc[df_filtered['temp'] < df_filtered.groupby('device').temp.transform(lambda x: x.quantile(.99))]
df_filtered = df_filtered.loc[df_filtered['humidity'] > df_filtered.groupby('device').humidity.transform(lambda x: x.quantile(.01))]
df_filtered = df_filtered.loc[df_filtered['humidity'] < df_filtered.groupby('device').humidity.transform(lambda x: x.quantile(.99))]

In [14]:
# group by device
# df_filtered = df_filtered.loc[df_filtered['device'] == 'iot-demo-device-01']
groups = df_filtered.groupby('device')

In [15]:
print('Filtered DataFrame Range')
print('-------------')
print('Record count: {:,}'.format(df_filtered['temp'].count()))
print('-------------')
print('Time range (min): {:%Y-%m-%d %H:%M:%S %Z}'.format(df_filtered.index[1]))
print('Time range (max): {:%Y-%m-%d %H:%M:%S %Z}'.format(df_filtered.index[-1]))
print('Temperature (min): {:.2f}'.format(df_filtered['temp'].min()))
print('Temperature (max): {:.2f}'.format(df_filtered['temp'].max()))
print('Humidity (min): {:.2f}{}'.format(df_filtered['humidity'].min(), '%'))
print('Humidity (max): {:.2f}{}'.format(df_filtered['humidity'].max(), '%'))
print('-------------')
print('Record count:\n{}'.format(groups.size()))
print('Temperature (min):\n{}'.format(groups['temp'].min()))
print('Temperature (max):\n{}'.format(groups['temp'].max()))
print('Humidity (min)\n{}'.format(groups['humidity'].min()))
print('Humidity (max):\n{}'.format(groups['humidity'].max()))

Filtered DataFrame Range
-------------
Record count: 0
-------------


IndexError: index 1 is out of bounds for axis 0 with size 0

In [16]:
# matplotlib datetime config
plt.rcParams['timezone']=MY_TIMEZONE
register_matplotlib_converters()
myFmt = DateFormatter(DATETIME_FORMAT)

In [ ]:
_, ax = plt.subplots(1, 1, figsize=(18, 9))
for device, group in groups:
    ax.plot(group.temp,
            group.humidity,
            marker='o',
            linestyle='',
            alpha=.5,
            ms=10,
            label=device)
ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('Temperature vs. Humidity')
plt.xlabel('Temperature (˚F)')
plt.ylabel('Humidity (%)')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 9))
for device, group in groups:
    ax.plot(group.temp,
            label=device)
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(myFmt)
ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('Temperature')
plt.ylabel('Temperature (˚F)')
plt.xlabel('Time')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 9))
for device, group in groups:
    ax.plot(group.humidity,
            label=device)
fig.autofmt_xdate()
ax.xaxis.set_major_formatter(myFmt)
ax.grid()
ax.margins(0.05)
ax.legend()
plt.title('Humidity')
plt.ylabel('Humidity (%)')
plt.xlabel('Time')
plt.show()

In [ ]:
# strip timezone info so Plotly won't convert to UTC
df_filtered.index = df_filtered.index.tz_localize(None)

fig = px.scatter(df_filtered,
                 x='temp',
                 y='humidity',
                 color='device',
                 hover_name='device',
                 hover_data={'device': False,
                             'temp':':.2f',
                             'humidity':':.2f'})

fig.update_layout(title='Temperature vs. Humidity',
                  xaxis_title='Temperature (˚F)',
                  yaxis_title='Humidity (%)',
                  template='ggplot2')

fig.show()

In [ ]:
fig = px.line(df_filtered,
              x=df_filtered.index.values,
              y='temp',
              color='device',
              hover_name='device',
              hover_data={'device': False,
                             'temp':':.2f'})

fig.update_layout(title='Temperature',
                  xaxis_title='Date/Time',
                  yaxis_title='Temperature (˚F)',
                  template='ggplot2')

fig.show()

In [ ]:
fig = px.line(df_filtered,
              x=df_filtered.index.values,
              y='humidity',
              color='device',
              hover_name='device',
              hover_data={'device': False,
                          'humidity':':.2f'})

fig.update_layout(title='Humidity',
                  xaxis_title='Time',
                  yaxis_title='Humidity (%)',
                  template='ggplot2')

fig.show()

In [ ]:
fig = px.scatter(df_filtered,
                 x=df_filtered.index.values,
                 y='light',
                 color='device')

fig.update_layout(title='Humidity',
                  xaxis_title='Time',
                  yaxis_title='Humidity (%)',
                  template='ggplot2')

fig.show()